In [2]:
#Import libraries
import bs4
import requests
import pandas as pd 
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import string


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\APOORV\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\APOORV\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Scrape data

In [3]:
issues = ['1906','1905','1904','1903','1902','1901',
          '1806','1805','1804','1803','1802','1801',
          '1706','1705','1704','1703','1702','1701',
          '1612', '1611', '1610', '1609', '1607', '1606', '1605', '1604', '1603', '1601',
          '1512', '1511', '1510', '1509', '1507', '1506', '1505', '1504', '1503', '1501',
          '1412', '1411', '1410', '1409', '1407', '1406', '1405', '1404', '1403', '1401',
          '1312', '1311', '1310', '1309', '1307', '1306', '1305', '1304', '1303', '1301',
          '1212', '1211', '1210', '1209', '1207', '1206', '1205', '1204', '1203', '1201'
          ]
data = pd.DataFrame(columns = ['Year','Issue', 'Article Name', 'Author', 'Content', 'link'])
for f in tqdm(issues):
    url = "https://hbr.org/archive-toc/BR"+f
    cpage = requests.get(url)
    csoup = bs4.BeautifulSoup(cpage.content, 'lxml')

    #Find links to all the articles

    raw = csoup.find_all('h3', {'class':'hed'})
    raw = set(raw)
    s = []
    for i in raw:
        try:
            a = i.find('a').attrs['href']
            s.append(a)
        except:
            m = 1
    for i in range(len(s)):
        s[i] = 'http://hbr.org'+s[i]


    #Iterate throught the articles for the issue


    
    k = -1
    for i in s:
        page = requests.get(i)
        soup = bs4.BeautifulSoup(page.content, 'lxml')

        content = soup.find(name = 'article-content')
        try:
            title = content.find('h1', {'class':'article-hed'}).text
        except:
            title = content.find_all('h2', {'class':'article-hed'})[k].text
            if len(content.find_all('h2', {'class':'article-hed'}))>1:
                    k+=1

        try:
            author = content.find('div').find('div').find('div').find('div', {"class": "byline"}).find('ul').find('li').text
        except:
            author = ''
        contents = ''
        body = content.find('div', {'article article-first-row premium'})
        if(body==None):
            try:
                body = content.find_all('div', {'class':'article article-first-row'})[k]
            except:
                body = content.find_all('div', {'class':'article article-first-row'})
        body = body.find_all('p')
        for j in body:
            contents+= j.text + ' '
        try:
            issue = body.find('div', {'class': 'mvm description-text hbrgray, line-height-tight'}).find('a').text
        except:
            m = 2
        t = '20' + f[0:2]
        s2 = pd.Series([t, f, title, author, contents, i], index=['Year','Issue', 'Article Name', 'Author', 'Content', 'link'])
        data = data.append(s2, ignore_index=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 68/68 [19:58<00:00, 17.62s/it]


In [4]:
data

,Year,Issue,Article Name,Author,Content,link
0,2019,1906,How the Best Bosses Interrupt Bias on Their Teams,Joan C. Williams,Companies spend millions on antibias training ...,http://hbr.org/2019/11/how-the-best-bosses-int...
1,2019,1906,The CEO’s Guide to Retirement,,"When Jensen Huang cofounded NVIDIA, in 1993, h...",http://hbr.org/2019/11/the-ceo-life-cycle
2,2019,1906,The Truth About Open Offices,,Organizations are turning to open offices and ...,http://hbr.org/2019/11/the-truth-about-open-of...
3,2019,1906,Cracking the Code of Sustained Collaboration,,"In all too many organizations, efforts to prom...",http://hbr.org/2019/11/cracking-the-code-of-su...
4,2019,1906,How Should We Measure the Digital Economy?,,Though we spend a large and growing share of o...,http://hbr.org/2019/11/how-should-we-measure-t...
...,...,...,...,...,...,...
1424,2012,1201,Why “Good Jobs” Are Good for Retailers,Zeynep Ton,Almost one-fifth of American workers have bad ...,http://hbr.org/2012/01/why-good-jobs-are-good-...
1425,2012,1201,A New Approach to Funding Social Enterprises,Antony Bugg-Levine,The economic crisis of 2008 deeply damaged the...,http://hbr.org/2012/01/a-new-approach-to-fundi...
1426,2012,1201,Avoid the Traps That Can Destroy Family Busine...,"George Stalk, Jr.",Nearly 75 years ago a charismatic Brazilian en...,http://hbr.org/2012/01/avoid-the-traps-that-ca...
1427,2012,1201,Generation Robot,Jimmy Guterman,Sit down with some big technology thinkers and...,http://hbr.org/2012/01/generation-robot


In [5]:
data_19 = data[data['Year']=='2019']
data_18 = data[data['Year']=='2018']
data_17 = data[data['Year']=='2017']
data_16 = data[data['Year']=='2016']
data_15 = data[data['Year']=='2015']
data_14 = data[data['Year']=='2014']
data_13 = data[data['Year']=='2013']
data_12 = data[data['Year']=='2012']

In [6]:
text = ''
for i, j in data.iterrows():
    text += j[2] + ' '
arttext = ''
for i, j in data.iterrows():
    arttext += j[4]+ ' '


Tokenize and process

In [7]:
arttext_tokens = word_tokenize(arttext.lower())
text_tokens = word_tokenize(text.lower())

In [8]:
title_tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]

In [9]:
tokens_without_sw = [word for word in arttext_tokens if not word in stopwords.words()]

In [10]:
unique_words = set(title_tokens_without_sw)

In [11]:
unique_words = set(title_tokens_without_sw)
art_counts = []
title_counts = []
for words in unique_words :
    art_counts.append(tokens_without_sw.count(words))
for words in unique_words :
    title_counts.append(title_tokens_without_sw.count(words))

art_search = []
title_search = []
text = text.replace("“", '"')
text = text.replace("”", '"')
text = text.replace("’", "'")
text = text.replace("—", "-")

text = text.replace("\'", "'")

for i in unique_words:
    r = ''
    for sentence in text.split('. '):
        if i in sentence.lower():
            r+=sentence + '; '
    
    art_search.append(r)
for i in unique_words:
    r = ''
    for sentence in arttext.split('. '):
        if i in sentence.lower():
            r+=sentence + '; '
    
    title_search.append(r)


In [12]:
word_counts = pd.DataFrame(unique_words, columns = ['word'])
word_counts['Title counts'] = title_counts
word_counts['Article counts'] = art_counts
word_counts['Article text'] = title_search
word_counts['Title text'] = art_search

Remove certain stopwords from the dataframe

In [13]:
stopwords = ['would', 'could', 'well', 'need', 'less', 'said', 'see', 'us', 'might', 'much', 'two', 'get',
            'us', 'many', 'even','’', '“', '”', 'new', 'like', 'good', 'may', 'help' 'three',
            'use', 'others', 'found', 'another',
             'however','long', 'call', 'large', 'used', 'four', 'although', 'makes'] 
for i, j in word_counts.iterrows():
    if j[0] in string.punctuation:
        word_counts = word_counts.drop(i)
    if j[0] in stopwords:
        word_counts = word_counts.drop(i)


In [14]:
r = word_counts.sort_values(by=['Title counts'], ascending = False).head(500)


In [15]:
r

,word,Title counts,Article counts,Article text,Title text
1071,work,89,6320,The goal is to create workforces that are more...,How the Best Bosses Interrupt Bias on Their Te...
1628,interview,73,322,Another found that men from elite backgrounds ...,How the Best Bosses Interrupt Bias on Their Te...
1412,ceo,68,2633,One look at NVIDIA’s stock chart shows how thi...,How the Best Bosses Interrupt Bias on Their Te...
1925,life,63,1097,"Whenever possible, stick to working hours, or ...",How the Best Bosses Interrupt Bias on Their Te...
587,case,46,1253,Using sensors and digital data to track intera...,How the Best Bosses Interrupt Bias on Their Te...
...,...,...,...,...,...
987,spark,2,62,It’s also a way of asking questions so as to s...,Facebook's Dream of World Domination CEOs with...
1677,negotiation,2,252,Although the program was designed for internal...,Navy Life's Work: An Interview with Alice Wate...
2316,transformation,2,397,These transformation efforts have the potentia...,Economy? Beating the Odds Better Brainstorming...
50,bet,2,88,Studies show that well-managed diverse groups ...,How the Best Bosses Interrupt Bias on Their Te...


In [34]:
def counter(word, data, k):
    t = ''
    l = 0
    for l in data[k]:
        t += l + ' '
    tokens = [word for word in word_tokenize(t.lower())]
    
    return tokens.count(word)
    
    

In [35]:
def usage(r, k):
    usage = []
    for i,j in tqdm(r.iterrows()):
        word = j[0]
        s = [counter(word, data_12, k),
            counter(word, data_13, k),
            counter(word, data_14, k),
            counter(word, data_15, k),
            counter(word, data_16, k),
            counter(word, data_17, k),
            counter(word, data_18, k),
            counter(word, data_19, k)]
        usage.append(s)
    return(usage)

In [36]:
r['usage in titles'] = usage(r, 'Article Name')

500it [00:22, 21.76it/s]


In [58]:
s = pd.read_csv("F:/Playin/HBR Data.csv")
r['usage in articles'] = d

In [44]:
d = []
for i in s['yearwise']:
    d.append(i)

array('[876, 894, 971, 743, 707, 643, 753, 733]', dtype='<U40')

Dump data to a csv file for accessing through R.

In [60]:
r.to_csv("F:/HBR data.csv")

In [64]:
data.to_csv("F:/HBR Article Listing.csv")

In [59]:
r

,word,Title counts,Article counts,Article text,Title text,usage in titles,usage in articles
1071,work,89,6320,The goal is to create workforces that are more...,How the Best Bosses Interrupt Bias on Their Te...,"[7, 14, 16, 14, 12, 10, 6, 10]","[876, 894, 971, 743, 707, 643, 753, 733]"
1628,interview,73,322,Another found that men from elite backgrounds ...,How the Best Bosses Interrupt Bias on Their Te...,"[2, 5, 11, 24, 13, 6, 6, 6]","[29, 47, 63, 40, 44, 33, 34, 31]"
1412,ceo,68,2633,One look at NVIDIA’s stock chart shows how thi...,How the Best Bosses Interrupt Bias on Their Te...,"[9, 8, 8, 10, 9, 10, 6, 8]","[268, 315, 366, 279, 402, 345, 448, 209]"
1925,life,63,1097,"Whenever possible, stick to working hours, or ...",How the Best Bosses Interrupt Bias on Their Te...,"[4, 4, 14, 11, 10, 6, 7, 7]","[168, 158, 172, 120, 132, 92, 109, 146]"
587,case,46,1253,Using sensors and digital data to track intera...,How the Best Bosses Interrupt Bias on Their Te...,"[0, 1, 2, 10, 13, 6, 8, 6]","[196, 203, 172, 181, 203, 102, 107, 90]"
...,...,...,...,...,...,...,...
987,spark,2,62,It’s also a way of asking questions so as to s...,Facebook's Dream of World Domination CEOs with...,"[0, 1, 0, 1, 0, 0, 0, 0]","[115, 241, 151, 98, 93, 74, 82, 85]"
1677,negotiation,2,252,Although the program was designed for internal...,Navy Life's Work: An Interview with Alice Wate...,"[0, 0, 0, 2, 0, 0, 0, 0]","[80, 73, 73, 29, 46, 66, 52, 32]"
2316,transformation,2,397,These transformation efforts have the potentia...,Economy? Beating the Odds Better Brainstorming...,"[0, 0, 1, 0, 0, 1, 0, 0]","[33, 13, 33, 25, 10, 7, 18, 6]"
50,bet,2,88,Studies show that well-managed diverse groups ...,How the Best Bosses Interrupt Bias on Their Te...,"[0, 1, 0, 0, 0, 1, 0, 0]","[304, 248, 282, 260, 217, 157, 173, 141]"
